In [3]:
!pip install selenium

  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.5 MB 8.4 MB/s eta 0:00:02
   --------------- ------------------------ 3.7/9.5 MB 10.9 MB/s eta 0:00:01
   --------------------------- ------------ 6.6/9.5 MB 11.8 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.5 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 11.4 MB/s eta 0:00:00
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached wspro

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\sandr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [31]:
# Set up Chrome options
path_to_extension = "C:/Users/sandr/AppData/Local/Google/Chrome/User Data/Default/Extensions/cjpalhdlnbpafiamejdnhcphjbkeiagm/1.62.0_1"

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument(f'--load-extension={path_to_extension}')

# Initialize Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Define the base URLs for each deck list page
urls = [
    "https://onepiecetopdecks.com/deck-list/english-format-op1-and-st1to4-meta-decks/",
    "https://onepiecetopdecks.com/deck-list/en-format-op02-paramount-war-decklist/",
    "https://onepiecetopdecks.com/deck-list/en-format-op03-mighty-enemy-decklist/",
    "https://onepiecetopdecks.com/deck-list/en-format-op04-kingdom-of-intrigue-decklist/",
    "https://onepiecetopdecks.com/deck-list/en-format-op05-awakening-of-the-new-era/",
    "https://onepiecetopdecks.com/deck-list/en-format-op-06-wings-of-the-captain-decks/",
    "https://onepiecetopdecks.com/deck-list/english-eb-01-memorial-set-op-07-500-years-into-the-future-decks/"
]

def dismiss_consent_modal():
    try:
        wait = WebDriverWait(driver, 5)  # Wait max 5 seconds for modal to appear
        do_not_consent_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.fc-button.fc-cta-do-not-consent"))
        )
        do_not_consent_button.click()
        print("Dismissed consent modal.")
        time.sleep(2)  # Allow time for modal to disappear
    except Exception:
        print("No consent modal detected.")

def get_deck_links(driver):
    deck_links = []
    a_tags = driver.find_elements(By.TAG_NAME, 'a')
    for a_tag in a_tags:
        href = a_tag.get_attribute('href')
        if '/deck-list/' in href and 'deckgen' in href:
            deck_links.append(href)
    print(f"Found {len(deck_links)} deck links.")
    return deck_links

def parse_deck(deck_url):
    try:
        # driver.get(deck_url)
        # time.sleep(3)
        deck_data = deck_url.split('&dg=')[1].split('&cs=')[0]
        
        # Parse the deck data string
        deck_items = []
        for item in deck_data.split('a'):
            if item:
                count = int(item.split('n')[0])
                card_code = item.split('n')[1]
                deck_items.append({card_code: count})
                
        return deck_items
    except Exception as e:
        print(f"Error parsing deck from url: {deck_url}")
        return []

def scrape_decks(base_url):
    deck_links = []
    driver.get(base_url)

    time.sleep(3)
    dismiss_consent_modal()
    
    while True:
        time.sleep(3)
        deck_links.extend(get_deck_links(driver))
        
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button.dt-paging-button.next")
            driver.implicitly_wait(5)
            print(next_button.get_attribute('class'))
            if 'disabled' in next_button.get_attribute('class'):
                break
            print("Navigating to next page...")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5) # lazy loading
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1) # lazy loading

            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
        except Exception as e:
            print(f"Error navigating to next page: {e}")
            break
        time.sleep(1)
    
    return deck_links


In [32]:
all_deck_lists = []

for url in urls:
    print(f"Scraping deck lists from {url}...")
    deck_links = scrape_decks(url)
    for deck_link in deck_links:
        # print(f"Scraping deck data from {deck_link}...")
        deck_data = parse_deck(deck_link)
        if deck_data:
            all_deck_lists.append(deck_data)
        # time.sleep(1)

print("Scraping complete.")

Scraping deck lists from https://onepiecetopdecks.com/deck-list/english-format-op1-and-st1to4-meta-decks/...
Dismissed consent modal.
Found 100 deck links.
dt-paging-button next
Navigating to next page...
Found 70 deck links.
dt-paging-button disabled next
Scraping deck lists from https://onepiecetopdecks.com/deck-list/en-format-op02-paramount-war-decklist/...
No consent modal detected.
Found 100 deck links.
dt-paging-button next
Navigating to next page...
Found 77 deck links.
dt-paging-button disabled next
Scraping deck lists from https://onepiecetopdecks.com/deck-list/en-format-op03-mighty-enemy-decklist/...
No consent modal detected.
Found 100 deck links.
dt-paging-button next
Navigating to next page...
Found 100 deck links.
dt-paging-button next
Navigating to next page...
Found 14 deck links.
dt-paging-button disabled next
Scraping deck lists from https://onepiecetopdecks.com/deck-list/en-format-op04-kingdom-of-intrigue-decklist/...
No consent modal detected.
Found 100 deck links.


In [34]:


# Save the collected deck lists to a JSON file
with open('data/one_piece_deck_lists.json', 'w') as f:
    json.dump(all_deck_lists, f, indent=2)